In [1]:
from collab_v1.data_higgs import load_data, download_and_make_data
import pandas as pd
from sklearn.feature_selection import SelectFromModel
import tensorflow as tf

from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
from solution import get_training_data, train_catboost, train_dnn, draw_loss, roc_plots, normal_model, overfitting_model
import numpy

2025-07-22 15:15:33.445659: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753190133.457922   13823 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753190133.461456   13823 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753190133.471942   13823 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753190133.471965   13823 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753190133.471967   13823 computation_placer.cc:177] computation placer alr

In [2]:
# download_and_make_data()
dataset = load_data()
X_train, y_train, X_val, y_val = get_training_data(dataset)
print(X_train.head())
print("SHAPE", X_train.shape, X_val.shape)
dataset.close()

Loading /home/luka/Documents/Programming/school/mp/ml/data/higgs-parsed/higgs-parsed.h5...
Loaded.
   lepton-pT  lepton-eta  lepton-phi  missing-energy  missing-energy-phi  \
0   1.138683   -0.726635   -0.005790        0.204118            0.153842   
1   0.404633    1.014821   -1.050041        1.136441           -1.403536   
2   1.137585    0.325251    1.453598        0.804114            0.893516   
3   1.380438   -0.595149   -0.727112        0.465392           -0.057453   
4   0.962628    1.191110   -1.161568        1.541759            0.569159   

   jet_1-pt  jet_1-eta  jet_1-phi  jet_1-b-tag  jet_2-pt  ...  jet_4-eta  \
0  1.585904  -0.045576  -1.448527     1.086538  1.598471  ...  -2.439800   
1  3.218436  -1.944837   0.801788     0.000000  2.238942  ...  -1.174742   
2  0.418095  -1.164536  -0.585919     0.000000  0.653565  ...   0.280201   
3  0.399224  -0.076273   1.080084     2.173076  0.644878  ...   1.261267   
4  1.337374   0.810973   0.458075     1.086538  0.549946  ...   

In [8]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd


# Start H2O
h2o.init()

# Convert to H2OFrames
train_h2o = h2o.H2OFrame(pd.concat([X_train, y_train], axis=1))
valid_h2o = h2o.H2OFrame(pd.concat([X_val, y_val], axis=1))

# Define target and features
target = y_train.name
features = [col for col in train_h2o.columns if col != target]
train_h2o[target] = train_h2o[target].asfactor()
valid_h2o[target] = valid_h2o[target].asfactor()




Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "21.0.7" 2025-04-15; OpenJDK Runtime Environment (Red_Hat-21.0.7.0.6-2) (build 21.0.7+6); OpenJDK 64-Bit Server VM (Red_Hat-21.0.7.0.6-2) (build 21.0.7+6, mixed mode, sharing)
  Starting server from /home/luka/.venvs/tf/lib/python3.11/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpszh85fd8
  JVM stdout: /tmp/tmpszh85fd8/h2o_luka_started_from_python.out
  JVM stderr: /tmp/tmpszh85fd8/h2o_luka_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Amsterdam
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,3 months and 6 days
H2O_cluster_name:,H2O_from_python_luka_1nnycz
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.703 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Run AutoML
aml = H2OAutoML(max_models=20, seed=1, balance_classes=True, nfolds=5)
aml.train(x=features, y=target, training_frame=train_h2o, validation_frame=valid_h2o)

AutoML progress: |
22:48:08.745: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),9/20
# GBM base models (used / total),3/7
# XGBoost base models (used / total),5/6
# DRF base models (used / total),0/2
# DeepLearning base models (used / total),1/4
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [ ]:
print(aml.leaderboard)
lb_df = aml.leaderboard.as_data_frame()

for index, row in lb_df.iterrows():
    print(row['model_id'], row['auc'])

model_id                                                      auc    logloss     aucpr    mean_per_class_error      rmse       mse
StackedEnsemble_AllModels_1_AutoML_1_20250702_224808     0.827362   0.506606  0.841777                0.275882  0.411375  0.16923
StackedEnsemble_BestOfFamily_1_AutoML_1_20250702_224808  0.824502   0.510198  0.839238                0.279653  0.413078  0.170633
GBM_1_AutoML_1_20250702_224808                           0.821975   0.513459  0.836827                0.281283  0.414562  0.171862
GBM_4_AutoML_1_20250702_224808                           0.820815   0.515139  0.835319                0.283638  0.415213  0.172402
GBM_3_AutoML_1_20250702_224808                           0.818954   0.517782  0.833378                0.286677  0.416355  0.173352
GBM_2_AutoML_1_20250702_224808                           0.817276   0.519884  0.831743                0.291496  0.41734   0.174173
XGBoost_grid_1_AutoML_1_20250702_224808_model_2          0.815101   0.522386  0.8296

/home/luka/Documents/Programming/school/mp/ml/.venv/lib64/python3.11/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [ ]:
best_ensemble = aml.leader  # usually the best model is the stacked ensemble

# List base model IDs used by the stacked ensemble
base_model_ids = best_ensemble.base_models
print(base_model_ids)

# Optionally, print model types or other info for each base model
for mid in base_model_ids:
    model = h2o.get_model(mid)
    print(f"Model ID: {model.model_id}, Algorithm: {model.algo}")

['GBM_1_AutoML_1_20250702_224808', 'GBM_4_AutoML_1_20250702_224808', 'GBM_3_AutoML_1_20250702_224808', 'GBM_2_AutoML_1_20250702_224808', 'XGBoost_grid_1_AutoML_1_20250702_224808_model_2', 'GBM_5_AutoML_1_20250702_224808', 'XGBoost_3_AutoML_1_20250702_224808', 'XGBoost_grid_1_AutoML_1_20250702_224808_model_3', 'XGBoost_grid_1_AutoML_1_20250702_224808_model_1', 'XGBoost_2_AutoML_1_20250702_224808', 'GBM_grid_1_AutoML_1_20250702_224808_model_2', 'XRT_1_AutoML_1_20250702_224808', 'DRF_1_AutoML_1_20250702_224808', 'GBM_grid_1_AutoML_1_20250702_224808_model_1', 'XGBoost_1_AutoML_1_20250702_224808', 'DeepLearning_1_AutoML_1_20250702_224808', 'DeepLearning_grid_1_AutoML_1_20250702_224808_model_1', 'DeepLearning_grid_3_AutoML_1_20250702_224808_model_1', 'DeepLearning_grid_2_AutoML_1_20250702_224808_model_1', 'GLM_1_AutoML_1_20250702_224808']
Model ID: GBM_1_AutoML_1_20250702_224808, Algorithm: gbm
Model ID: GBM_4_AutoML_1_20250702_224808, Algorithm: gbm
Model ID: GBM_3_AutoML_1_20250702_224808,

In [ ]:
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators import H2OXGBoostEstimator, H2ORandomForestEstimator, H2OGradientBoostingEstimator 

# Define hyperparameters to tune
hyper_params = {
    'ntrees': [50, 100, 150, 200],
    'max_depth': [5, 8, 11, 14],
}

def get_grid(Estimator):
    # Instantiate base model
    xgb = Estimator(seed=42)

    # Grid search object
    grid = H2OGridSearch(model=xgb, hyper_params=hyper_params)

    grid.train(x=features, y=target, training_frame=train_h2o, validation_frame=valid_h2o)


    return grid




In [ ]:
xg_grid = get_grid(H2OXGBoostEstimator)

xgboost Grid Build progress: |███████████████████████████████████████████████████| (done) 100%


In [ ]:
rdg_grid = get_grid(H2ORandomForestEstimator)

drf Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%


In [ ]:
gbm_grid = get_grid(H2OGradientBoostingEstimator) 

gbm Grid Build progress: |███████████████████████████████████████████████████████Job request failed Unexpected HTTP error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), will retry after 3s.
Job request failed Unexpected HTTP error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), will retry after 3s.


In [5]:
import fcntl
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from joblib import Parallel, delayed
import os

save_path = 'aucs.npy'
if os.path.exists(save_path):
    aucs = np.load(save_path)
else:
    aucs = np.full_like(X, fill_value=np.nan, dtype=float)
np.save(save_path, aucs)  # Ensure it exists

def write_auc(i, j, auc):
    with open(save_path, 'rb+') as f:
        fcntl.flock(f, fcntl.LOCK_EX)
        aucs = np.load(f)
        aucs[i, j] = auc
        f.seek(0)
        np.save(f, aucs)
        fcntl.flock(f, fcntl.LOCK_UN)

def compute_and_store_auc(i, j):
    ntrees = int(X[i, j])
    max_depth = int(Y[i, j])
    gbm = GradientBoostingClassifier(
        random_state=42,
        n_estimators=ntrees,
        max_depth=max_depth
    )
    gbm.fit(X_train, y_train)
    y_prob = gbm.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_prob)
    write_auc(i, j, auc)

to_evaluate = [(i, j) for i in range(X.shape[0]) for j in range(X.shape[1]) if np.isnan(aucs[i, j])]

Parallel(n_jobs=-1, backend='loky')(
    delayed(compute_and_store_auc)(i, j) for i, j in to_evaluate
)

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Build DataFrame from H2O models
grid = gbm_grid.get_grid(sort_by='auc', decreasing=True)
grid



,max_depth,ntrees,model_ids,auc
,10.0,107.0,Grid_GBM_py_3_sid_bbce_model_python_1751546849245_4295_model_1,0.8235033
,10.0,107.0,Grid_GBM_py_3_sid_bbce_model_python_1751546849245_4295_model_10,0.8235033
,10.0,107.0,Grid_GBM_py_3_sid_bbce_model_python_1751546849245_4295_model_15,0.8235033
,10.0,100.0,Grid_GBM_py_3_sid_bbce_model_python_1751546849245_4295_model_8,0.8231082
,15.0,250.0,Grid_GBM_py_3_sid_bbce_model_python_1751546849245_4295_model_5,0.8228879
,15.0,200.0,Grid_GBM_py_3_sid_bbce_model_python_1751546849245_4295_model_3,0.8217187
,15.0,150.0,Grid_GBM_py_3_sid_bbce_model_python_1751546849245_4295_model_11,0.8205502
,15.0,100.0,Grid_GBM_py_3_sid_bbce_model_python_1751546849245_4295_model_2,0.8191126
,10.0,50.0,Grid_GBM_py_3_sid_bbce_model_python_1751546849245_4295_model_7,0.8178728
,15.0,50.0,Grid_GBM_py_3_sid_bbce_model_python_1751546849245_4295_model_9,0.8158602


In [5]:
max_depths = [5, 10, 15]
ntrees = [50, 100, 150, 200, 250]
scores = []

for max_depth in max_depths:
    for ntree in ntrees:
        model = CatBoostClassifier(iterations=ntree, verbose=100, depth=max_depth, task_type="GPU")
        model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)
        y_pred = model.predict(X_val)
        auc = roc_auc_score(y_val, y_pred)
        scores.append((auc, max_depth, ntree))
        print(max_depth, ntree)

Learning rate set to 0.119327
0:	learn: 0.6761734	test: 0.6760731	best: 0.6760731 (0)	total: 58.9ms	remaining: 2.88s
49:	learn: 0.5604931	test: 0.5601533	best: 0.5601533 (49)	total: 2.86s	remaining: 0us
bestTest = 0.5601533125
bestIteration = 49
5 50
Learning rate set to 0.08956
0:	learn: 0.6801780	test: 0.6801001	best: 0.6801001 (0)	total: 59ms	remaining: 5.84s
99:	learn: 0.5512450	test: 0.5510300	best: 0.5510300 (99)	total: 5.74s	remaining: 0us
bestTest = 0.55103
bestIteration = 99
5 100
Learning rate set to 0.07572
0:	learn: 0.6820888	test: 0.6820249	best: 0.6820249 (0)	total: 59.2ms	remaining: 8.82s
100:	learn: 0.5548104	test: 0.5546252	best: 0.5546252 (100)	total: 5.78s	remaining: 2.8s
149:	learn: 0.5461731	test: 0.5460393	best: 0.5460393 (149)	total: 8.58s	remaining: 0us
bestTest = 0.5460393125
bestIteration = 149
5 150
Learning rate set to 0.067218
0:	learn: 0.6832813	test: 0.6832237	best: 0.6832237 (0)	total: 58.9ms	remaining: 11.7s
100:	learn: 0.5577437	test: 0.5574745	best: 0

In [7]:
print("max_depth ntrees auc")

for x in sorted(scores, reverse=True):
    print(f'{x[1]} {x[2]} {x[0]}')

max_depth ntrees auc
15 250 0.7499667451149331
15 200 0.7486118489398761
15 150 0.7465574174911689
15 100 0.7433970174145756
10 250 0.7389531799773611
15 50 0.7379715317392941
10 200 0.7375975169263765
10 150 0.7355002789759607
10 100 0.7320728067619978
10 50 0.7264600840478392
5 250 0.7216250529162054
5 200 0.7198353710094519
5 150 0.7180127893289111
5 100 0.714820956203413
5 50 0.708824892035232


In [ ]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

dl = H2ODeepLearningEstimator(hidden=[512, 256, 128, 64], epochs=50)
dl.train(x=features, y=target, training_frame=train_h2o, validation_frame=valid_h2o)


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1751483365269_2


Status of Neuron Layers: predicting hlabel, 2-class classification, bernoulli distribution, CrossEntropy loss, 187,458 weights/biases, 2.2 MB, 7,798,861 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate             rate_rms               momentum    mean_weight            weight_rms           mean_bias             bias_rms
--  -------  -------  ---------  ---------  ----  ----  --------------------  ---------------------  ----------  ---------------------  -------------------  --------------------  -------------------
    1        28       Input      0.0
    2        512      Rectifier  0.0        0.0   0.0   0.01127217091830143   0.004291437566280365   0.0         -0.021651601734470116  0.39146339893341064  -0.6211932077729012   0.3244746923446655
    3        256      Rectifier  0.0        0.0   0.0   0.017919173506766084  0.015566065907478333   0.0         -0.03599794097371323   0.12138453125953674  0.8076219441267025    0.4042240381240845
    4        128      Rectifier  0.0        0.0   0.0   0.01455436159532475   0.014854144304990768   0.0         -0.020993453697453535  0.11634767055511475  0.4146723887349901    0.4686603546142578
    5        64       Rectifier  0.0        0.0   0.0   0.03225691694927413   0.030837245285511017   0.0         -0.032842726084992446  0.12655872106552124  -0.2948056842786058   0.3760782480239868
    6        2        Softmax               0.0   0.0   0.002788184706332686  0.0016877162270247936  0.0         -0.0952036228554789    0.5596945285797119   -0.05732492552489305  0.14778268337249756

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.15910692495499054
RMSE: 0.39888209405160135
LogLoss: 0.4810602121647156
Mean Per-Class Error: 0.25288857079485144
AUC: 0.8482786257233698
AUCPR: 0.8584006784661996
Gini: 0.6965572514467395

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.42528111673839936
       0     1     Error    Rate
-----  ----  ----  -------  ----------------
0      3016  1798  0.3735   (1798.0/4814.0)
1      697   4572  0.1323   (697.0/5269.0)
Total  3713  6370  0.2474   (2495.0/10083.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.425281     0.785635  237
max f2                       0.186283     0.871445  326
max f0point5                 0.657515     0.788853  146
max accuracy                 0.54573      0.763067  189
max precision                0.998518     1         0
max recall                   0.000617754  1         399
max specificity              0.998518     1         0
max absolute_mcc             0.54573      0.525591  189
max min_per_class_accuracy   0.548102     0.762568  188
max mean_per_class_accuracy  0.54573      0.762955  189
max tns                      0.998518     4814      0
max fns                      0.998518     5252      0
max fps                      0.000617754  4814      399
max tps                      0.000617754  5269      399
max tnr                      0.998518     1         0
max fnr                      0.998518     0.996774  0
max fpr                      0.000617754  1         399
max tpr                      0.000617754  1         399

Gains/Lift Table: Avg response rate: 52.26 %, avg score: 53.80 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  ----------------